*The maps in this notebook will not be shown on GitHub, but it is possible to see the rendered notebook here:* https://nbviewer.org/github/tkatus/demos/blob/main/Crime_London.ipynb

# Crime in London

The Metropolitan Police reports monthly updated crime figures for London's boroughs. 

In [1]:
import pandas as pd
import folium
import re

url = 'https://data.london.gov.uk/download/recorded_crime_summary/d2e9ccfc-a054-41e3-89fb-53c2bc3ed87a/MPS%20Borough%20Level%20Crime%20%28most%20recent%2024%20months%29.csv' 
cDF = pd.read_csv(url)
cDF.set_index('LookUp_BoroughName', inplace = True)
cDF.head()

,MajorText,MinorText,202005,202006,202007,202008,202009,202010,202011,202012,...,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204
LookUp_BoroughName,,,,,,,,,,,,,,,,,,,,,
Barking and Dagenham,Arson and Criminal Damage,Arson,2,4,4,6,2,7,4,2,...,4,12,5,7,6,1,4,4,3,5
Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,86,120,123,114,116,120,100,110,...,130,143,111,126,109,116,126,110,117,126
Barking and Dagenham,Burglary,Burglary Business and Community,16,16,28,23,32,20,18,24,...,21,29,27,37,20,14,19,29,30,18
Barking and Dagenham,Burglary,Domestic Burglary,42,63,72,63,54,68,90,91,...,61,87,62,82,87,91,81,67,76,83
Barking and Dagenham,Drug Offences,Drug Trafficking,17,10,21,10,12,14,18,13,...,6,9,15,17,9,11,15,11,19,13


GPS locations of London's boroughs are publicly available on wikipedia. 

In [2]:
def londonBuroughs():
    url = r'https://en.wikipedia.org/wiki/List_of_London_boroughs'
    tempo = pd.read_html(url) # , match = 'List of boroughs and local authorities')    
    # add city of london to list 
    d = tempo[0]
    tempo[1].columns = d.columns        # the two tables have slightly different columns, so for now use the columsn of the main table
    d.loc[len(d)+1] = tempo[1].iloc[0]   # add city of london

# Cleaning the data
    # remove square and round brackets
    df = d['Borough'].str.extract(r'([\w ]*)')
    df.columns = ['boroughs']
    df['local authority'] = d['Local authority'].str.extract(r'([\w ]*)')
    df['political control'] = d['Political control'].str.extract(r'([\w]*)') # use single word
    df['area'] = d['Area (sq mi)']
    df['population'] = d['Population (2019 est)[1]']
        
# parsing GPS coordinates
    def convert2GPS(thisCoord):
        tempo = float(thisCoord[0][0])
        if thisCoord[0][1] in ['S', 'W']:  # make negative if East or South
            tempo = -1 * tempo 
        return tempo

    lats = []; lngs=[]
    for idx, row in d.iterrows():
        x = re.search(r'\ufeff\d', row['Co-ordinates']) # find second GPS coordinate (1st one is no good)
        x = row['Co-ordinates'][x.span()[1]-1:]
        lng = re.findall(r'([0-9.]+)°([NS]{1})',x)
        lat = re.findall(r'([0-9.]+)°([EW]{1})',x)        
        lats.append(convert2GPS(lat))
        lngs.append(convert2GPS(lng))

    df['longitude'] = lngs
    df['latitude'] = lats
    df.set_index('boroughs', inplace=True)
    return df

bDF = londonBuroughs()
print('DataFrame bDF lists the boroughs of London, the GPS coordinates of their HQs and some basic demographic information (e.g., population)')
bDF.head()


DataFrame bDF lists the boroughs of London, the GPS coordinates of their HQs and some basic demographic information (e.g., population)


,local authority,political control,area,population,longitude,latitude
boroughs,,,,,,
Barking and Dagenham,Barking and Dagenham London Borough Council,Labour,13.93,212906,51.5607,0.1557
Barnet,Barnet London Borough Council,Labour,33.49,395896,51.6252,-0.1517
Bexley,Bexley London Borough Council,Conservative,23.38,248287,51.4549,0.1505
Brent,Brent London Borough Council,Labour,16.70,329771,51.5588,-0.2817
Bromley,Bromley London Borough Council,Conservative,57.97,332336,51.4039,0.0198


------------

__Basic map__ Plot borough population in an interactive pop-up map. 

In [3]:
centerGPSCoord = (51.509865, -0.15) 

m = folium.Map(location = centerGPSCoord, zoom_start=11)
for idx, row in bDF.iterrows():
    if idx == 'City of London':
        col = 'black'
    else:
        col = 'blue'
        
    folium.CircleMarker(
        fill_color='white',
        radius = 10,         
        location = [row.longitude, row.latitude],        
        color=col, 
        opacity = 1,
        fill_opacity = .3,
        fill=True, 
        popup =  idx + ' ' + str(round(row.population/1000)) + 'k').add_to(m)
       
display(m)

_________
__Crime map__ Plot crime per capita in map.

In [4]:
print('DataFrame cDF reports crime stats per borough. Offences reported by the MET police:\n')
print(sorted(list(set(cDF['MinorText']))))

DataFrame cDF reports crime stats per borough. Offences reported by the MET police:

['Absconding from Lawful Custody', 'Aggravated Vehicle Taking', 'Aiding Suicide', 'Arson', 'Bail Offences', 'Bicycle Theft', 'Bigamy', 'Burglary Business and Community', 'Criminal Damage', 'Dangerous Driving', 'Disclosure, Obstruction, False or Misleading State', 'Domestic Burglary', 'Drug Trafficking', 'Exploitation of Prostitution', 'Forgery or Use of Drug Prescription', 'Fraud or Forgery Associated with Driver Records', 'Going Equipped for Stealing', 'Handling Stolen Goods', 'Homicide', 'Interfering with a Motor Vehicle', 'Making, Supplying or Possessing Articles for use i', 'Obscene Publications', 'Offender Management Act', 'Other Firearm Offences', 'Other Forgery', 'Other Knife Offences', 'Other Notifiable Offences', 'Other Offences Against the State, or Public Order', 'Other Sexual Offences', 'Other Theft', 'Perjury', 'Perverting Course of Justice', 'Possession of Article with Blade or Point', 'P

In [5]:
# CHANGE THIS VARIABLE to look at a different type of crime
thisCrime = 'Theft or Taking of a Motor Vehicle'  
# --------------------------------------------------------
markerSizes = [5, 30]

a = cDF[cDF['MinorText'] == thisCrime][cDF.columns[-1]]                   # last column has the data from the most recent month   
c = bDF.merge(a, how='left', left_on='boroughs', right_on='LookUp_BoroughName', right_index=True) # allow missing values as long as there are GPS coordinates
c.columns = list(c.columns[:-1]) + ['crimes']
c['perCapita'] = c.crimes / c.population

# get colors to represent ruling party
def party2col(x): # political control
    cm = []
    for zed in x:
        if zed == 'Labour':
            cm.append('red')
        elif zed == 'Conservative':
            cm.append('blue')
        elif zed == 'Liberal':
            cm.append('orange')
        elif zed == 'Green':
            cm.append('green')
        else: 
            cm.append('gray')
    return cm

# map perCapita crimes so that min and max values match range determined by markerSizes
def scaleValues(x, markerSizes):
    # scale variable range to 0-1
    x = (x - x.min())
    x = x / x.max()    
    # scale to min - max markerSizes
    return x * (max(markerSizes) - min(markerSizes)) + min(markerSizes)

c['markerCol'] = party2col(bDF['political control'])
c['markerSize'] = scaleValues(c['perCapita'], markerSizes)
c.head()

# PLOT -------
c = c.dropna()
centerGPSCoord = (51.509865, -0.15) 

m = folium.Map(location = centerGPSCoord, zoom_start=11)
for idx, row in c.iterrows():        
    folium.CircleMarker(
        fill_color='white',
        radius = row.markerSize,         
        location = [row.longitude, row.latitude],        
        color = row.markerCol, 
        opacity = 1,
        fill_opacity = .3,
        fill=True, 
        popup =  idx + ' pop = ' + str(round(row.population/1000)) + 'k crimes = ' + str(int(row.crimes))).add_to(m)

print('Crime: ', thisCrime)
display(m)  

Crime:  Theft or Taking of a Motor Vehicle


The sizes of the circles indicate crimes per Capita. Their colours indicate the currently ruling party (red for Labour, blue for Tories, orange for LibDems, green for Green, gray for other). 


----------------
__TBC__ (work in progress) - Choropleth is next